# Classification of Land Use with the Masterful CLI Trainer

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)][1]&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[![Download](images/download.png)][2][Download this Notebook][2]

[1]:https://colab.research.google.com/github/masterfulai/masterful-docs/blob/main/notebooks/guide_cli_classification_land_use.ipynb
[2]:https://docs.masterfulai.com/0.5.2/notebooks/guide_cli_classification_land_use.ipynb

In this guide, you will take a deeper look at Classification with the [Masterful CLI Trainer](../notebooks/tutorial_quickstart_cli.ipynb) to train a state of the art model. 

This guide will use a satellite imagery dataset from [UCMerced](http://weegee.vision.ucmerced.edu/datasets/landuse.html) to classify different plots of land according to their usage. 

## Define the Application

In this guide, your challenge is to build a computer vision model that can classify different plots of land according to their usage. In this instance, the images have already been extracted from larger satellite imagery and each image can be classified into a single category.

Since each image can be categorized into a single class, and there are more than one possible classes, the computer vision task for this problem is **multi-class classification**.


## TL;DR

Don't want to read the rest of this guide, and want to start training
immediately? The following command shows you how to start training
with Masterful, using a configuration file and dataset on S3.

```
masterful-train --config https://masterful-public.s3.us-west-1.amazonaws.com/datasets/ucmerced/training.yaml
```

## Coming Soon